In [85]:
# Benchmark LogAI with open datasets

In [86]:
import pandas as pd
from logai.dataloader.data_loader import DataLoaderConfig, FileDataLoader

filepath = "/Users/qcheng/workspace/gitsoma/logai/logai/data/open_datasets/HealthApp.log"
#filepath = "./data/sf_datasets/dbscan_clustering_clean - dbscan_clustering_clean.csv"
log_type = "log"
# dimensions = {'attributes': ['cluster_label'],
#               'body': ['logline']}

log_type = "log"
dimensions = None

file_config = DataLoaderConfig(
    filepath=filepath,
    log_type=log_type,
    sep = '|'
)

dataloader = FileDataLoader(file_config)
logrecord = dataloader.load_data()

In [87]:
from logai.dataloader.data_model import LogRecordObject

df = pd.read_csv(filepath, sep="|", on_bad_lines="warn", header=None)
df.columns=["Timestamps", "Action", "ID", "Details"]

df_selected = df.head(1000)


# 20171223-22:15:29:606
#
logrecord = LogRecordObject(
    timestamp=pd.to_datetime(df_selected['Timestamps'], format="%Y%m%d-%H:%M:%S:%f"),
    attributes=df_selected[['Action', 'ID']],
    body=df_selected[['Details']]
)

b'Skipping line 1794: expected 4 fields, saw 7\nSkipping line 8472: expected 4 fields, saw 7\nSkipping line 17280: expected 4 fields, saw 7\nSkipping line 37868: expected 4 fields, saw 7\nSkipping line 53628: expected 4 fields, saw 7\nSkipping line 66532: expected 4 fields, saw 7\nSkipping line 66979: expected 4 fields, saw 7\nSkipping line 87622: expected 4 fields, saw 7\nSkipping line 102000: expected 4 fields, saw 7\nSkipping line 116461: expected 4 fields, saw 7\n'
b'Skipping line 138346: expected 4 fields, saw 7\nSkipping line 138857: expected 4 fields, saw 7\nSkipping line 174408: expected 4 fields, saw 7\nSkipping line 179394: expected 4 fields, saw 7\nSkipping line 189076: expected 4 fields, saw 7\nSkipping line 195498: expected 4 fields, saw 7\nSkipping line 208961: expected 4 fields, saw 7\nSkipping line 226413: expected 4 fields, saw 7\nSkipping line 238798: expected 4 fields, saw 7\nSkipping line 249895: expected 4 fields, saw 7\n'


In [88]:
# import os
#
# w_path = "/Users/qcheng/workspace/gitsoma/logai/tests/logai/test_data"
#
# logrecord.attributes.to_pickle(os.path.join(w_path, "healthapp_attributes"))
# logrecord.timestamp.to_pickle(os.path.join(w_path, "healthapp_timestamp"))

#

In [89]:
# Describe data
print(logrecord.body.shape)
logrecord.body.head(5)

(1000, 1)


,Details
0,onStandStepChanged 3579
1,onExtend:1514038530000 14 0 4
2,onReceive action: android.intent.action.SCREEN_ON
3,processHandleBroadcastAction action:android.in...
4,flush sensor data


In [90]:
from logai.algorithms.parsing_algo.ael import AELParams
from logai.algorithms.parsing_algo.iplom import IPLoMParams
from logai.information_extraction.log_parser import LogParserConfig, LogParser
from logai.algorithms.parsing_algo.drain import DrainParams

# Information Extraction: Parsing
# Currently only implemented DRAIN. IPLoM next but needs time to refactor code from LogPAI.

loglines = logrecord.body['Details']

drain_params = DrainParams(
    sim_th=0.4
)

iplom_params = IPLoMParams()

ael_params = AELParams()

log_parser_config = LogParserConfig(
    parsing_algorithm='drain',
    parsing_algo_params=drain_params
)

parser = LogParser(log_parser_config)
parsed_result = parser.parse(loglines)

In [91]:
from logai.information_extraction.log_vectorizer import VectorizerConfig, LogVectorizer
from logai.algorithms.vectorization_algo.word2vec import Word2VecParams

## Vectorization

parsed_loglines = parsed_result['parsed_logline']

# Vectorization using Word2Vec

params = Word2VecParams()
vectorizer_config = VectorizerConfig("word2vec", params, None)
vectorizor = LogVectorizer(vectorizer_config)
vectorizor.fit(parsed_loglines)

#Log vector is a pandas.Series
log_vectors_w2v = vectorizor.transform(parsed_loglines)

In [92]:
from logai.information_extraction.categorical_encoder import CategoricalEncoderConfig, CategoricalEncoder

encoder_config = CategoricalEncoderConfig(name="label_encoder")

encoder = CategoricalEncoder(encoder_config)

attributes = encoder.fit_transform(logrecord.attributes)

attributes.head(5)

,Action_categorical,ID_categorical
0,11,0
1,11,0
2,14,0
3,11,0
4,15,0


In [93]:
from logai.information_extraction.feature_extractor import FeatureExtractor, FeatureExtractorConfig
#Feature extraction
# implement log vector to feature
# this will convert the vector metrics to a n dimensional feature.
# implement simple 0 padding method.


import pandas as pd

config = FeatureExtractorConfig()
feature_extractor = FeatureExtractor(config)
_, feature_vector = feature_extractor.convert_to_feature_vector(log_vectors_w2v, attributes, logrecord.timestamp)

In [94]:
feature_vector.head(5)

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_92,feature_93,feature_94,feature_95,feature_96,feature_97,feature_98,feature_99,Action_categorical,ID_categorical
0,-0.125446,0.246017,-0.051116,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0
1,0.215296,0.299100,-0.167181,-0.251936,-0.034387,0.325121,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0
2,-0.297246,-0.234719,0.030049,0.192571,-0.325141,0.209281,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,0.0
3,-0.052588,0.010712,-0.138021,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0
4,0.092888,0.229703,0.203370,-0.103328,0.263144,0.197952,-0.214433,0.022391,0.297987,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0


In [95]:
from logai.analysis.clustering import ClusteringConfig, Clustering
# Log clustering
#Clustering using K-Means


from logai.algorithms.clustering_algo.kmeans import KMeansParams

# algo_params = KMeansParams(n_clusters=100)
clustering_config = ClusteringConfig('DBSCAN')

clustering = Clustering(clustering_config)

clustering.fit(feature_vector)
kmeans_res = clustering.predict(feature_vector)
res_df = pd.concat((kmeans_res.rename('cluster_label'), parsed_result), axis=1)

In [96]:
res_df[res_df['cluster_label']==1].head(5)

,cluster_label,logline,parsed_logline,parameter_list
1,1,onExtend:1514038530000 14 0 4,* * 0 4,"[onExtend:1514038530000, 14]"
11,1,onExtend:1514038530000 0 0 4,* * 0 4,[onExtend:1514038530000]
14,1,onExtend:1514038530000 1 0 4,* * 0 4,"[onExtend:1514038530000, 1]"
21,1,onExtend:1514038531000 1 0 4,* * 0 4,"[onExtend:1514038531000, 1]"
28,1,onExtend:1514038531000 2 0 4,* * 0 4,"[onExtend:1514038531000, 2]"


In [97]:
res_df.cluster_label.value_counts()

 0     152
 3     123
 5     123
 6     123
 1     111
 7     109
 4     100
-1      91
 8      26
 9      17
 10     14
 2      11
Name: cluster_label, dtype: int64

In [98]:
#Anomaly Detection

from sklearn.model_selection import train_test_split

feature_for_anomaly_detection = feature_vector
train, test = train_test_split(feature_for_anomaly_detection, train_size=0.7, test_size=0.3)

In [99]:
from logai.algorithms.anomaly_detection_algo.one_class_svm import OneClassSVMParams
from logai.analysis.anomaly_detector import AnomalyDetectionConfig, AnomalyDetector

algo_params = OneClassSVMParams()

config = AnomalyDetectionConfig(algo_name='one_class_svm')

anomaly_detector = AnomalyDetector(config)
anomaly_detector.fit(train)
res = anomaly_detector.predict(test)

[LibSVM]*
optimization finished, #iter = 217
obj = 4810626.676652, rho = 34127.711071
nSV = 350, nBSV = 350


In [103]:
pd.concat((parsed_result.iloc[res[res==1].index], res[res==1].rename('anomaly_label')), axis=1).head(5)

,logline,parsed_logline,parameter_list,anomaly_label
56,onExtend:1514038534000 1 0 4,* * 0 4,"[onExtend:1514038534000, 1]",1
367,getTodayTotalDetailSteps = 1514038560000##706...,getTodayTotalDetailSteps = *,[1514038560000##7064##548557##8661##16256##272...,1
210,REPORT : 7035 5022 150689 240,REPORT : * * * 240,"[7035, 5022, 150689]",1
609,REPORT : 7122 5085 152553 240,REPORT : * * * 240,"[7122, 5085, 152553]",1
672,REPORT : 7154 5107 153238 240,REPORT : * * * 240,"[7154, 5107, 153238]",1


In [104]:
config = FeatureExtractorConfig(
    group_by_category=["parsed_logline", "Action", "ID"]
)

feature_extractor = FeatureExtractor(config)

counter_vector = feature_extractor.convert_to_counter_vector(
    log_pattern=parsed_loglines,
    attributes=logrecord.attributes,
    timestamps=logrecord.timestamp
)

In [102]:
counter_vector.head(5)



,parsed_logline,Action,ID,Counts
0,* * 0 4,Step_LSC,30002312,111
1,* 0 0 *,Step_LSC,30002312,26
2,* size = * = *,HiH_HiHealthDataInsertStore,30002312,3
3,* the * is true,HiH_,30002312,4
4,"* the * switch is open, start push * data!",HiH_HiSyncControl,30002312,4
